# Uso de las bases de datos

La interacción con las bases de datos puede hacerse con distintos objetivos.

Cada una de estas tareas puede ser realizadas por un bioinformático,
pero requiere distintos perfiles de conocimiento:

- Realizar un análisis de datos de una vez.
- Automatizar un análisis periódico.
- Desarrollar una aplicación que utiliza y/o requiere datos a pedido de un
  usuario.



## Como obtener datos de las Bases de datos biológicas

Las formas más frecuentes de obtener información de las bases de datos son:

- Acceso por una página Web:
  - Útil para la exploración de pocos datos.
  - Interacción fácil con otras bases de datos
  - Suele estar bien documentada.
  - Puede ser suficiente para obtener datos para hacer un análisis en particular.
  - Recuperar grandes cantidades de datos puede ser tedioso y lento.
  - No se puede automatizar.

- Acceso FTP u otro método de descarga masiva de datos:
  - La mayoría de los proveedores permiten la descarga de los datos crudos, en paquetes grandes que contienen la totalidad o grandes porciones de los datos disponibles.
  - Suele haber poca documentación.
  - Útil para hacer análisis a gran escala, que requiere todos los datos disponibles.
  - No permite obtener datos particulares.
  - Puede resultar útil si se está desarrollando una aplicación que se actualiza en períodos de tiempo largo.

- Acceso programático:
  - Útil para la automatización de tareas y la creación de aplicaciones.
  - Poca documentación.
  - Restricciones para evitar el abuso.
  - Poco eficiente para descarga masiva de datos.
  - Requiere más conocimientos informáticos y de programación.



En esta parte del curso nos vamos en el acceso programático.
Para ello, tenemos que entender algunos conceptos básicos de como se transmiten los datos en internet.

## Protocolo HTTP

El HTTP (Hypertext Transfer Protocol, Procolo de transferencia de hipertexto) es un protocolo para la comunicación de datos de cualquier medio, de forma distribuida y colaborativa.

En este protocolo hay dos actores principales:

- El **cliente** (client): Es quien hace la petición de conexión y hace los **pedidos** (request) al servidor.
- El **servidor** (server): Es quien recibe los pedidos del cliente y devuelve una **respuesta** (response) a ese pedido.

![](https://raw.githubusercontent.com/javieriserte/bases-de-datos-unq-unnoba/main/images/http_scheme.svg)

### Principios del protocolo HTTP

- Sin conexión permanente:

El **cliente** comienza una conexión mediante un **pedido**. El cliente espera al **servidor** que tiene que procesar
el **pedido** y luego envía la **respuesta** al **cliente**. Cuando el **cliente** recibe la **respuesta** la conexión termina y tanto el **cliente** y el **servidor** dejan de tener conocimiento de cada uno.

- Sin estado:

Ni el **cliente** ni el **servidor** pueden mantener información entre distintas conexiones.

- No tiene restricciones de los medios que se transfieren.

Se puede transferir datos de cualquier tipo, siempre que el **cliente** y el **servidor** sepan como interpretarlos.

## Pedidos y respuestas HTTP

Los **pedidos** y las **respuestas** HTTP tienen una estructura fija y contienen datos definidos.

### Pedidos HTTP:

Los pedidos contienen dos partes.

- La primera, llamada **encabezado** (**header**), contiene datos que definen el tipo de pedido y el destinatario.
Los elementos más importantes del encabezado son:
  - Método: Define el tipo acción que debe realizar el **servidor** al recibir el pedido. Los dos métodos más comunes son:
    - GET: Para recuperar información del servidor.
    - POST: Para subir datos que requerirá el servidor para procesar el **pedido**.
  - Recurso: Ruta del recurso (por ejemplo un archivo al que se dirige el **pedido**).
  - Host: Nombre del **servidor**.
- La segunda parte son los datos que se envian al servidor. No es obligatoria.

<pre>
<b style="color:red">POST</b> /<b style="color:blue">Blast.cgi</b> HTTP/2
<b style="color:green">Host: blast.ncbi.nlm.nih.gov</b>
User-Agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:88.0) Gecko/20100101 Firefox/88.0
Accept: text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8
Accept-Language: es-AR,es;q=0.8,en-US;q=0.5,en;q=0.3
Accept-Encoding: gzip, deflate, br
Referer: https://blast.ncbi.nlm.nih.gov/Blast.cgi
Content-Type: application/x-www-form-urlencoded
Content-Length: 1000
Origin: https://blast.ncbi.nlm.nih.gov
Connection: keep-alive
Cookie: ncbi_sid=8D9FBA5
Upgrade-Insecure-Requests: 1
Sec-GPC: 1

<b style="color:purple">ADV_VIEW=on&ALIGNMENTS=100&ALIGNMENT_VIEW=Pairwise</b>
</pre>

### Respuestas HTTP:

Las repuestas tienen un formato similar, pero no van a ser relevantes para nosotros, con exceptión del código de estado.

El código de estado (**status code**) es un número que refleja si la consulta fue exitosa o si hubo algún tipo de problema.

Algunos de los códigos de estados más comunes son:

- 200 OK : La consulta se procesó exitosamente.
- 301 Moved Permanently: La url no se encuentra en el lugar indicado, la nueva url se pasa en la respuesta.
- 400 Bad Request: El servidor no entiende el pedido.
- 404 Not found: La url no se encontró.
- 500 Internal Server Error: El servidor encontró una situación que no sabe como manejar.

## REST API

Los servidores webs pueden ofrecer sus datos y permitir hacer consultas
por medio de una interfaz programática independiente de la representación
de una página web.

Este tipo de interfaz se conoce como REST-API.

- De esta forma es posible interactuar con el servidor de forma más simple.

- Esta interfaz está conformada por una colección de **URLs** diferentes.

- Cada una de ellas provee una funcionalidad específica.

- Por ejemplo, estas son algunas de las **URLs** de NCBI.

  - https://eutils.ncbi.nlm.nih.gov/entrez/eutils/einfo.fcgi
  - https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi
  - https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esummary.fcgi

- Como programadores, podemos interactuar con esta interfaz para llevar las
  tareas que necesitemos.

- La comunicación se realiza por medio de HTTP.

## Como interactuar con una REST-API de una base de datos.

La interacción con una REST-API puede hacerse de muchas formas.

Nosotros nos vamos a restringir a dos casos típicos usando el lenguage de programación R.

- Usando la librería httr, que permite hacer consultas de forma genérica a
  servicios http.
- Usando librería específicas para interactuar con alguna base de datos
  particular.

### Usando la librería httr de R

Es un mecanismo básico para hacer peticiones HTTP genéricas.

Nosotros tendremos que encargarnos de adaptar los datos a los formatos que sean necesarios y de procesar las respuestas.

Suele requerir un manejo más técnico y de más detalles que otras soluciones.

In [2]:
# Para instalar la libraría
# install.packages("httr")

library(httr)
library(xml2)

Vamos a comenzar haciendo un pedido con el método GET.

In [1]:
# Defino la URL
url <- "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/einfo.fcgi"

In [3]:
# La función GET envía la petición y devuelve la respuesta del servidor
response <- GET(url)

In [4]:
# Veamos el status code
status_code(response)

[1] 200

In [5]:
# Veamos el contenido de la respuesta
# Primero como texto plano
contenido <- content(response, as="text")
cat(contenido)

<?xml version="1.0" encoding="UTF-8" ?>
<!DOCTYPE eInfoResult PUBLIC "-//NLM//DTD einfo 20190110//EN" "https://eutils.ncbi.nlm.nih.gov/eutils/dtd/20190110/einfo.dtd">
<eInfoResult>
<DbList>

	<DbName>pubmed</DbName>
	<DbName>protein</DbName>
	<DbName>nuccore</DbName>
	<DbName>ipg</DbName>
	<DbName>nucleotide</DbName>
	<DbName>structure</DbName>
	<DbName>genome</DbName>
	<DbName>annotinfo</DbName>
	<DbName>assembly</DbName>
	<DbName>bioproject</DbName>
	<DbName>biosample</DbName>
	<DbName>blastdbinfo</DbName>
	<DbName>books</DbName>
	<DbName>cdd</DbName>
	<DbName>clinvar</DbName>
	<DbName>gap</DbName>
	<DbName>gapplus</DbName>
	<DbName>grasp</DbName>
	<DbName>dbvar</DbName>
	<DbName>gene</DbName>
	<DbName>gds</DbName>
	<DbName>geoprofiles</DbName>
	<DbName>homologene</DbName>
	<DbName>medgen</DbName>
	<DbName>mesh</DbName>
	<DbName>nlmcatalog</DbName>
	<DbName>omim</DbName>
	<DbName>orgtrack</DbName>
	<DbName>pmc</DbName>
	<DbName>popset</DbName>
	<DbName>proteinclusters</DbName>
	<DbNa

In [6]:
# Veamos el contenido de la respuesta
# Ahora, transformando el contenido a una lista de R
contenido <- as_list(content(response, as="parsed"))
class(contenido)
contenido

[1] "list"

$eInfoResult
$eInfoResult$DbList
$eInfoResult$DbList$DbName
$eInfoResult$DbList$DbName[[1]]
[1] "pubmed"


$eInfoResult$DbList$DbName
$eInfoResult$DbList$DbName[[1]]
[1] "protein"


$eInfoResult$DbList$DbName
$eInfoResult$DbList$DbName[[1]]
[1] "nuccore"


$eInfoResult$DbList$DbName
$eInfoResult$DbList$DbName[[1]]
[1] "ipg"


$eInfoResult$DbList$DbName
$eInfoResult$DbList$DbName[[1]]
[1] "nucleotide"


$eInfoResult$DbList$DbName
$eInfoResult$DbList$DbName[[1]]
[1] "structure"


$eInfoResult$DbList$DbName
$eInfoResult$DbList$DbName[[1]]
[1] "genome"


$eInfoResult$DbList$DbName
$eInfoResult$DbList$DbName[[1]]
[1] "annotinfo"


$eInfoResult$DbList$DbName
$eInfoResult$DbList$DbName[[1]]
[1] "assembly"


$eInfoResult$DbList$DbName
$eInfoResult$DbList$DbName[[1]]
[1] "bioproject"


$eInfoResult$DbList$DbName
$eInfoResult$DbList$DbName[[1]]
[1] "biosample"


$eInfoResult$DbList$DbName
$eInfoResult$DbList$DbName[[1]]
[1] "blastdbinfo"


$eInfoResult$DbList$DbName
$eInfoResult$DbList$DbName[[1]]
[1] "books"


$eInfoResult$DbList$DbName
$eInfoResult$DbList$DbName[[1]]
[1] "cdd"


$eInfoResult$DbList$DbName
$eInfoResult$DbList$DbName[[1]]
[1] "clinvar"


$eInfoResult$DbList$DbName
$eInfoResult$DbList$DbName[[1]]
[1] "gap"


$eInfoResult$DbList$DbName
$eInfoResult$DbList$DbName[[1]]
[1] "gapplus"


$eInfoResult$DbList$DbName
$eInfoResult$DbList$DbName[[1]]
[1] "grasp"


$eInfoResult$DbList$DbName
$eInfoResult$DbList$DbName[[1]]
[1] "dbvar"


$eInfoResult$DbList$DbName
$eInfoResult$DbList$DbName[[1]]
[1] "gene"


$eInfoResult$DbList$DbName
$eInfoResult$DbList$DbName[[1]]
[1] "gds"


$eInfoResult$DbList$DbName
$eInfoResult$DbList$DbName[[1]]
[1] "geoprofiles"


$eInfoResult$DbList$DbName
$eInfoResult$DbList$DbName[[1]]
[1] "homologene"


$eInfoResult$DbList$DbName
$eInfoResult$DbList$DbName[[1]]
[1] "medgen"


$eInfoResult$DbList$DbName
$eInfoResult$DbList$DbName[[1]]
[1] "mesh"


$eInfoResult$DbList$DbName
$eInfoResult$DbList$DbName[[1]]
[1] "nlmcatalog"


$eInfoResult$DbList$DbName
$eInfoResult$DbList$DbName[[1]]
[1] "omim"


$eInfoResult$DbList$DbName
$eInfoResult$DbList$DbName[[1]]
[1] "orgtrack"


$eInfoResult$DbList$DbName
$eInfoResult$DbList$DbName[[1]]
[1] "pmc"


$eInfoResult$DbList$DbName
$eInfoResult$DbList$DbName[[1]]
[1] "popset"


$eInfoResult$DbList$DbName
$eInfoResult$DbList$DbName[[1]]
[1] "proteinclusters"


$eInfoResult$DbList$DbName
$eInfoResult$DbList$DbName[[1]]
[1] "pcassay"


$eInfoResult$DbList$DbName
$eInfoResult$DbList$DbName[[1]]
[1] "protfam"


$eInfoResult$DbList$DbName
$eInfoResult$DbList$DbName[[1]]
[1] "pccompound"


$eInfoResult$DbList$DbName
$eInfoResult$DbList$DbName[[1]]
[1] "pcsubstance"


$eInfoResult$DbList$DbName
$eInfoResult$DbList$DbName[[1]]
[1] "seqannot"


$eInfoResult$DbList$DbName
$eInfoResult$DbList$DbName[[1]]
[1] "snp"


$eInfoResult$DbList$DbName
$eInfoResult$DbList$DbName[[1]]
[1] "sra"


$eInfoResult$DbList$DbName
$eInfoResult$DbList$DbName[[1]]
[1] "taxonomy"


$eInfoResult$DbList$DbName
$eInfoResult$DbList$DbName[[1]]
[1] "biocollections"


$eInfoResult$DbList$DbName
$eInfoResult$DbList$DbName[[1]]
[1] "gtr"

Hagamos otra petición, pasando algunos argumentos adicionales.

In [7]:
url <- "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/einfo.fcgi"
# Según la documentación de EInfo en NCBI, el argumento "db" permite obtener detalles
# específicos de una base de datos particular.
response <- GET(url, query = list(db="nuccore"))

In [8]:
contenido <- content(response, as="text")
cat(contenido)


<?xml version="1.0" encoding="UTF-8" ?>
<!DOCTYPE eInfoResult PUBLIC "-//NLM//DTD einfo 20190110//EN" "https://eutils.ncbi.nlm.nih.gov/eutils/dtd/20190110/einfo.dtd">
<eInfoResult>
	<DbInfo>
	<DbName>nuccore</DbName>
	<MenuName>Nucleotide</MenuName>
	<Description>Core Nucleotide db</Description>
	<DbBuild>Build231020-2035m.1</DbBuild>
	<Count>609915146</Count>
	<LastUpdate>2023/10/23 02:36</LastUpdate>
	<FieldList>
		<Field>
			<Name>ALL</Name>
			<FullName>All Fields</FullName>
			<Description>All terms from all searchable fields</Description>
			<TermCount>13019157774</TermCount>
			<IsDate>N</IsDate>
			<IsNumerical>N</IsNumerical>
			<SingleToken>N</SingleToken>
			<Hierarchy>N</Hierarchy>
			<IsHidden>N</IsHidden>
		</Field>
		<Field>
			<Name>UID</Name>
			<FullName>UID</FullName>
			<Description>Unique number assigned to each sequence</Description>
			<TermCount>0</TermCount>
			<IsDate>N</IsDate>
			<IsNumerical>Y</IsNumerical>
			<SingleToken>Y</SingleToken>
			<Hierarchy>N</H

In [9]:
# Otra forma de hacerlo es modificar la URL, aunque es más engorroso y es más fácil equivocarnos.

url <- "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/einfo.fcgi?db=nuccore"
response <- GET(url)
cat(content(response, as="text"))

<?xml version="1.0" encoding="UTF-8" ?>
<!DOCTYPE eInfoResult PUBLIC "-//NLM//DTD einfo 20190110//EN" "https://eutils.ncbi.nlm.nih.gov/eutils/dtd/20190110/einfo.dtd">
<eInfoResult>
	<DbInfo>
	<DbName>nuccore</DbName>
	<MenuName>Nucleotide</MenuName>
	<Description>Core Nucleotide db</Description>
	<DbBuild>Build231020-2035m.1</DbBuild>
	<Count>609915146</Count>
	<LastUpdate>2023/10/23 02:36</LastUpdate>
	<FieldList>
		<Field>
			<Name>ALL</Name>
			<FullName>All Fields</FullName>
			<Description>All terms from all searchable fields</Description>
			<TermCount>13019157774</TermCount>
			<IsDate>N</IsDate>
			<IsNumerical>N</IsNumerical>
			<SingleToken>N</SingleToken>
			<Hierarchy>N</Hierarchy>
			<IsHidden>N</IsHidden>
		</Field>
		<Field>
			<Name>UID</Name>
			<FullName>UID</FullName>
			<Description>Unique number assigned to each sequence</Description>
			<TermCount>0</TermCount>
			<IsDate>N</IsDate>
			<IsNumerical>Y</IsNumerical>
			<SingleToken>Y</SingleToken>
			<Hierarchy>N</H

Ahora vamos probar con un POST

In [12]:
seq <- "MMASTENNEKDNFMRDTASRSKKSRRRSLWIAAGAVPTAIA\
LSLSLASPAAVAQSSFGSSDIIDSGVLDSITRGLTDYLTPR\
DEALPAGEVTYPAIEGLPAGVRVNSAEYVTSHHVVLSIQSA\
AMPERPIKVQLLLPRDWYSSPDRDFPEIWALDGLRAIEKQS\
GWTIETNIEQFFADKNAIVVLPVGGESSFYTDWNEPNNGKN\
YQWETFLTEELAPILDKGFRSNGERAITGISMGGTAAVNIA\
THNPEMFNFVGSFSGYLDTTSNGMPAAIGAALADAGGYNVN\
AMWGPAGSERWLENDPKRNVDQLRGKQVYVSAGSGADDYGQ\
DGSVATGPANAAGVGLELISRMTSQTFVDAANGAGVNVIAN\
FRPSGVHAWPYWQFEMTQAWPYMADSLGMSREDRGADCVAL\
GAIADATADGSLGSCLNNEYLVANGVGRAQDFTNGRAYWSP\
NTGAFGLFGRINARYSELGGPDSWLGFPKTRELSTPDGRGR\
YVHFENGSIYWSAATGPWEIPGDMFTAWGTQGYEAGGLGYP\
VGPAKDFNGGLAQEFQGGYVLRTPQNRAYWVRGAISAKYME\
PGVATTLGFPTGNERLIPGGAFQEFTNGNIYWSASTGAHYI\
LRGGIFDAWGAKGYEQGEYGWPTTDQTSIAAGGETITFQNG\
TIRQVNGRIEESR"

url <- "http://pfam-legacy.xfam.org/search/sequence"

body <- list(
    seq = seq,
    output="xml"
)
response <- POST(
    url,
    body=body
)

In [13]:
cat(content(response, as="text"))

<?xml version="1.0" encoding="UTF-8"?>
<jobs xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance"
      xmlns="https://pfam.xfam.org/"
      xsi:schemaLocation="https://pfam.xfam.org/
                          https://pfam.xfam.org/static/documents/schemas/submission.xsd">
  <job job_id="82DEA8DE-7291-11EE-9794-5E1EA594C059">
    <opened>2023-10-24T18:19:38</opened>
    <result_url>/search/sequence/resultset/82DEA8DE-7291-11EE-9794-5E1EA594C059?output=xml</result_url>
  </job>
</jobs>



### Usando librerías de terceros específicas para interactuar con alguna base de datos

Para muchas bases de datos, existen paquetes que permiten que la interacción son esa base de datos sea más sencilla.

Desafortunadamente, no existen para todas las bases de datos y no siempre son estables.
En general son desarrollados por una sola persona o equipos pequeños y no siempre están actualizados con
respecto a las bases de datos.

De todas formas, van a ser nuestra forma preferida de interacción por su sencillez.


In [16]:
# Para instalar
install.packages("rentrez")
library(rentrez)

also installing the dependency ‘XML’


Updating HTML index of packages in '.Library'

Making 'packages.html' ...
 done



In [62]:
response <- entrez_info()
response

[1] "<?xml version=\"1.0\" encoding=\"UTF-8\"?>\n<!DOCTYPE eInfoResult PUBLIC \"-//NLM//DTD einfo 20190110//EN\" \"https://eutils.ncbi.nlm.nih.gov/eutils/dtd/20190110/einfo.dtd\">\n<eInfoResult>\n  <DbList>\n    <DbName>pubmed</DbName>\n    <DbName>protein</DbName>\n    <DbName>nuccore</DbName>\n    <DbName>ipg</DbName>\n    <DbName>nucleotide</DbName>\n    <DbName>structure</DbName>\n    <DbName>genome</DbName>\n    <DbName>annotinfo</DbName>\n    <DbName>assembly</DbName>\n    <DbName>bioproject</DbName>\n    <DbName>biosample</DbName>\n    <DbName>blastdbinfo</DbName>\n    <DbName>books</DbName>\n    <DbName>cdd</DbName>\n    <DbName>clinvar</DbName>\n    <DbName>gap</DbName>\n    <DbName>gapplus</DbName>\n    <DbName>grasp</DbName>\n    <DbName>dbvar</DbName>\n    <DbName>gene</DbName>\n    <DbName>gds</DbName>\n    <DbName>geoprofiles</DbName>\n    <DbName>homologene</DbName>\n    <DbName>medgen</DbName>\n    <DbName>mesh</DbName>\n    <DbName>nlmcatalog</DbName>\n    <DbName>omim</DbName>\n    <DbName>orgtrack</DbName>\n    <DbName>pmc</DbName>\n    <DbName>popset</DbName>\n    <DbName>proteinclusters</DbName>\n    <DbName>pcassay</DbName>\n    <DbName>protfam</DbName>\n    <DbName>pccompound</DbName>\n    <DbName>pcsubstance</DbName>\n    <DbName>seqannot</DbName>\n    <DbName>snp</DbName>\n    <DbName>sra</DbName>\n    <DbName>taxonomy</DbName>\n    <DbName>biocollections</DbName>\n    <DbName>gtr</DbName>\n  </DbList>\n</eInfoResult>\n"

In [18]:
response <- entrez_info(db="nuccore")
response

<?xml version="1.0" encoding="UTF-8"?>
<!DOCTYPE eInfoResult PUBLIC "-//NLM//DTD einfo 20190110//EN" "https://eutils.ncbi.nlm.nih.gov/eutils/dtd/20190110/einfo.dtd">
<eInfoResult>
  <DbInfo>
    <DbName>nuccore</DbName>
    <MenuName>Nucleotide</MenuName>
    <Description>Core Nucleotide db</Description>
    <DbBuild>Build231020-2035m.1</DbBuild>
    <Count>609915146</Count>
    <LastUpdate>2023/10/23 02:36</LastUpdate>
    <FieldList>
      <Field>
        <Name>ALL</Name>
        <FullName>All Fields</FullName>
        <Description>All terms from all searchable fields</Description>
        <TermCount>13019157774</TermCount>
        <IsDate>N</IsDate>
        <IsNumerical>N</IsNumerical>
        <SingleToken>N</SingleToken>
        <Hierarchy>N</Hierarchy>
        <IsHidden>N</IsHidden>
      </Field>
      <Field>
        <Name>UID</Name>
        <FullName>UID</FullName>
        <Description>Unique number assigned to each sequence</Description>
        <TermCount>0</TermCount>
      